In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import json
import plotly.express as px

In [2]:
%pip3 install streamlit plotly

UsageError: Line magic function `%pip3` not found.


In [ ]:
ammos = pd.read_csv('./data/ammos.csv')
armors = pd.read_csv('./data/armors.csv')
ashes = pd.read_csv('./data/ashes.csv')
incantations = pd.read_csv('./data/incantations.csv')
sorceries = pd.read_csv('./data/sorceries.csv')
talismans = pd.read_csv('./data/talismans.csv')
classes = pd.read_csv('./data/classes.csv')
npcs = pd.read_csv('./data/npcs.csv')
weapons = pd.read_csv('./data/weapons.csv')
spirits = pd.read_csv('./data/spirits.csv')
creatures = pd.read_csv('./data/creatures.csv')
bosses = pd.read_csv('./data/bosses.csv')
df = pd.read_csv("./data/elden_ring_weapon.csv")

class EldenRing:

    def __init__(self, ammos, armors, ashes, incantations, sorceries, talismans, classes, npcs, weapons, spirits, creatures, bosses, df):
        self.ammos = ammos
        self.armors = armors
        self.ashes = ashes
        self.incantations = incantations
        self.sorceries = sorceries
        self.talismans = talismans
        self.classes = classes
        self.npcs = npcs
        self.weapons = weapons
        self.spirits = spirits
        self.creatures = creatures
        self.bosses = bosses
        self.df = df

    @staticmethod
    def search_item(items, name):
        return items[items['name'] == name]
        
    def plot_interactive_weapon_weight(self):
        self.weapons['customdata'] = self.weapons['name'] + ' - ' + self.weapons['category']

        fig = px.bar(self.weapons, x='name', y='weight', hover_data=['customdata'], title='Weapons')

        fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)'})
        fig.update_layout(font_color='white')

        fig.update_traces(hovertemplate='<b>Name:</b> %{customdata}<br><b>Weight:</b> %{y}<extra></extra>')

        fig.update_xaxes(title='Name')
        fig.update_yaxes(title='Weight')

        fig.show()

    def replaceZero(self):
        self.df = self.df.replace('-', 0)
        self.df['Phy'] = self.df['Phy'].astype(float)
        self.df['Mag'] = self.df['Mag'].astype(float)
        self.df['Hol'] = self.df['Hol'].astype(float)
        return self.df
    
    def groupByType(self): 
        average_physical_damage = self.df.groupby('Type')['Phy'].mean().reset_index()
        average_magic_damage = self.df.groupby('Type')['Mag'].mean().reset_index()
        average_holy_damage = self.df.groupby('Type')['Hol'].mean().reset_index()
        return average_physical_damage, average_magic_damage, average_holy_damage

    def plot_interactive_weapon_details(self, ph, mg, hl):
        average_damage = pd.concat([ph['Type'], ph['Phy'], mg['Mag'], hl['Hol']], axis=1)
        average_damage.columns = ['Type', 'Physical', 'Magic', 'Holy']

        average_damage['data'] = average_damage.apply(lambda row: f"Physical: {row['Physical']}<br>Magic: {row['Magic']}<br>Holy: {row['Holy']}", axis=1)
        
        fig = px.bar(average_damage, x='Type', y=['Physical', 'Magic', 'Holy'], barmode='group', hover_data=['data'])

        fig.update_traces(text=average_damage['Type'], textposition='outside')

        fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)'})
        fig.update_layout(font_color='white')
        
        fig.update_layout(
            title='Average Damage by Weapon Type',
            xaxis_title='Weapon Type',
            yaxis_title='Average Damage',
            legend_title='Damage Type'
        )

        # fig.update_layout(clickmode='event+select')
        
        # fig.add_trace(go.Scatter(x=average_damage['Type'], y=average_damage['Physical'], mode='markers', marker=dict(size=12), name='Physical'))
        # fig.add_trace(go.Scatter(x=average_damage['Type'], y=average_damage['Magic'], mode='markers', marker=dict(size=12), name='Magic'))
        # fig.add_trace(go.Scatter(x=average_damage['Type'], y=average_damage['Holy'], mode='markers', marker=dict(size=12), name='Holy'))
        
        return fig.show()

    def scatter_plot(self):
        sorted_data = self.df.sort_values(by='Phy', ascending=False)

        fig = px.scatter(sorted_data, x='Wgt', y='Phy', color='Type', hover_data=['Name', 'Wgt', 'Phy', 'Mag', 'Hol', 'Type'])

        fig.update_traces(marker=dict(size=12))

        fig.update_layout(title='Weight vs Physical Damage', xaxis_title='Weight', yaxis_title='Physical Damage')
        fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)'})
        fig.update_layout(font_color='white')

        fig.update_layout(clickmode='event+select')

        return fig.show()
    
    def incantation_plot(self):
        fig = px.bar(self.incantations, x='name', y='cost', title='Incantations')

        fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)'})
        fig.update_layout(font_color='white')

        fig.update_xaxes(title='Name')
        fig.update_yaxes(title='cost')

        return fig.show()
    
    def search_item(self, items, name):
        return items[items['name'] == name]


    def incantation_plot_requires(self):
        df = pd.DataFrame(columns=['name', 'cost', 'requires', 'requiresIntelligence', 'requiresFaith', 'requiresArcane'])

        list = []

        for name in self.incantations['name']:
            incantation = self.search_item(self.incantations, name)
            requires_value = incantation['requires'].iloc[0]
            requires = json.loads(requires_value.replace("'", '"')) if isinstance(requires_value, str) else []
            cost = incantation['cost'].values[0]

            intelligence_requirement = sum(req['amount'] for req in requires if req['name'] == 'Intelligence') / len(requires) if requires else 0
            faith_requirement = sum(req['amount'] for req in requires if req['name'] == 'Faith') / len(requires) if requires else 0
            arcane_requirement = sum(req['amount'] for req in requires if req['name'] == 'Arcane') / len(requires) if requires else 0

            list.append(pd.DataFrame({'name': [name], 'cost': [cost], 'requires': [requires], 'requiresIntelligence': [intelligence_requirement], 'requiresFaith': [faith_requirement], 'requiresArcane': [arcane_requirement]}))

        df = pd.concat(list)

        fig = px.scatter(df, x='requiresIntelligence', y='cost', title='Incantations', hover_data=['name'])

        # Ajoute une ligne de régression linéaire pour Requires Intelligence
        fig.add_traces(px.scatter(df, x='requiresIntelligence', y='cost', trendline="ols").data)

        # Ajoute une ligne de régression linéaire pour Requires Faith
        fig.add_traces(px.scatter(df, x='requiresFaith', y='cost', trendline="ols").data)

        # Ajoute une ligne de régression linéaire pour Requires Arcane
        fig.add_traces(px.scatter(df, x='requiresArcane', y='cost', trendline="ols").data)

        fig.update_layout(legend_title='Legend', xaxis_title='Requires', yaxis_title='Cost')

        fig.show()
game = EldenRing(ammos, armors, ashes, incantations, sorceries, talismans, classes, npcs, weapons, spirits, creatures, bosses, df)
df = game.replaceZero()

average_physical_damage, average_magic_damage, average_holy_damage = game.groupByType()

game.plot_interactive_weapon_details(average_physical_damage, average_magic_damage, average_holy_damage)

game.scatter_plot()

game.plot_interactive_weapon_weight()

game.incantation_plot_requires()